# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
ctry_df = pd.read_csv('../output_data/ctry_df.csv', index_col = False)
ctry_df.head(20)
ctry_df_data = ctry_df.dropna()
del ctry_df_data["Unnamed: 0"]
ctry_df_data.head(20)

,City,Date,Country,Max Temperature (F),Wind Speed (mph),Latitude,Longitude,Humidity (%),Cloudiness (%)
0,certeze,1.612650e+09,RO,39.470,1.06,47.9000,23.4667,100.0,90.0
1,kingsport,1.612650e+09,US,44.870,2.06,36.5484,-82.5618,45.0,90.0
2,saint-pierre,1.612650e+09,RE,79.070,2.06,-21.3393,55.4781,83.0,40.0
3,ushuaia,1.612650e+09,AR,53.870,8.75,-54.8000,-68.3000,71.0,40.0
4,cape town,1.612650e+09,ZA,72.266,4.12,-33.9258,18.4232,83.0,20.0
5,cherskiy,1.612650e+09,RU,-8.032,2.41,68.7500,161.3000,80.0,0.0
6,prince rupert,1.612650e+09,CA,39.470,2.50,54.3161,-130.3201,93.0,90.0
7,jamestown,1.612650e+09,US,21.470,4.63,42.0970,-79.2353,45.0,1.0
8,hermanus,1.612650e+09,ZA,67.262,1.63,-34.4187,19.2345,90.0,84.0
9,yellowknife,1.612650e+09,CA,-32.530,1.03,62.4560,-114.3525,61.0,1.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
# Conf gmaps
gmaps.configure(api_key=g_key)
# Locations
locations = ctry_df_data[["Latitude", "Longitude"]]
humidity = ctry_df_data["Humidity (%)"].astype(float)

# Set humidity
humidity = ctry_df_data["Humidity (%)"].astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
# Create criteria for the perfect vacation climate
# A max temperature lower than 80 degrees but higher than 65.
# Wind speed less than 10 and no clouds
# Filter for above conditions
i_wea_df = ctry_df_data[(ctry_df_data['Max Temperature (F)'] > 70) &
                        (ctry_df_data['Max Temperature (F)'] < 80) &
                        (ctry_df_data['Wind Speed (mph)'] < 10) &
                        (ctry_df_data['Cloudiness (%)'] == 0)]
# index to filter
i_wea_df = i_wea_df.dropna()
i_wea_df = i_wea_df.reset_index()
i_wea_df.head(10)

,index,City,Date,Country,Max Temperature (F),Wind Speed (mph),Latitude,Longitude,Humidity (%),Cloudiness (%)
0,38,maceio,1.612650e+09,BR,79.070,5.14,-9.6658,-35.7353,83.0,0.0
1,62,hobyo,1.612650e+09,SO,77.234,6.78,5.3505,48.5268,79.0,0.0
2,76,vao,1.612650e+09,NC,77.504,2.52,-22.6667,167.4833,85.0,0.0
3,85,chuy,1.612650e+09,UY,73.688,4.65,-33.6971,-53.4616,68.0,0.0
4,100,ponta pora,1.612650e+09,BR,79.070,3.60,-22.5361,-55.7256,44.0,0.0
5,198,guerrero negro,1.612650e+09,MX,72.806,4.29,27.9769,-114.0611,37.0,0.0
6,244,yulara,1.612650e+09,AU,75.470,4.63,-25.2406,130.9889,33.0,0.0
7,260,moratuwa,1.612650e+09,LK,77.270,2.57,6.7730,79.8816,88.0,0.0
8,298,ancud,1.612650e+09,CL,73.670,3.60,-41.8697,-73.8203,46.0,0.0
9,310,bafia,1.612650e+09,CM,72.266,0.58,4.7500,11.2333,78.0,0.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
# Store data into variable named `hotel_df`.
# Add a column to the DataFrame called 'Hotel Name'.
hotel_df = i_wea_df
hotel_df['Hotel Name'] = ''
hotel_df

,index,City,Date,Country,Max Temperature (F),Wind Speed (mph),Latitude,Longitude,Humidity (%),Cloudiness (%),Hotel Name
0,38,maceio,1.612650e+09,BR,79.070,5.14,-9.6658,-35.7353,83.0,0.0,
1,62,hobyo,1.612650e+09,SO,77.234,6.78,5.3505,48.5268,79.0,0.0,
2,76,vao,1.612650e+09,NC,77.504,2.52,-22.6667,167.4833,85.0,0.0,
3,85,chuy,1.612650e+09,UY,73.688,4.65,-33.6971,-53.4616,68.0,0.0,
4,100,ponta pora,1.612650e+09,BR,79.070,3.60,-22.5361,-55.7256,44.0,0.0,
5,198,guerrero negro,1.612650e+09,MX,72.806,4.29,27.9769,-114.0611,37.0,0.0,
6,244,yulara,1.612650e+09,AU,75.470,4.63,-25.2406,130.9889,33.0,0.0,
7,260,moratuwa,1.612650e+09,LK,77.270,2.57,6.7730,79.8816,88.0,0.0,
8,298,ancud,1.612650e+09,CL,73.670,3.60,-41.8697,-73.8203,46.0,0.0,
9,310,bafia,1.612650e+09,CM,72.266,0.58,4.7500,11.2333,78.0,0.0,


In [37]:
params = {"radius": 5000,"types": "lodging","keyword": "Hotel","key": g_key}

# Use the lat/lng to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from hotel_df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration
    params["location"] = f"{lat},{lng}"

    # URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    n_add = requests.get(base_url, params=params)
    
    #print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(n_add.url)

    # convert to json
    n_add = n_add.json()
    #print(json.dumps(n_add, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = n_add["results"][0]["name"]
#        hotel_df.loc[index, "Airport Address"] = n_add["results"][0]["vicinity"]
#        hotel_df.loc[index, "Airport Rating"] = n_add["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df.head()

Missing field/result... skipping.


,index,City,Date,Country,Max Temperature (F),Wind Speed (mph),Latitude,Longitude,Humidity (%),Cloudiness (%),Hotel Name
0,38,maceio,1.612650e+09,BR,79.070,5.14,-9.6658,-35.7353,83.0,0.0,Jatiúca Hotel & Resort
1,62,hobyo,1.612650e+09,SO,77.234,6.78,5.3505,48.5268,79.0,0.0,Osmani Hotel
2,76,vao,1.612650e+09,NC,77.504,2.52,-22.6667,167.4833,85.0,0.0,OURE TERA beach resort
3,85,chuy,1.612650e+09,UY,73.688,4.65,-33.6971,-53.4616,68.0,0.0,Turis Firper Hotel
4,100,ponta pora,1.612650e+09,BR,79.070,3.60,-22.5361,-55.7256,44.0,0.0,Hotel Pousada do Bosque


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [40]:
# Add marker layer ontop of heat map
fig_layout = {'width': '900px', 'height': '600px', 'border': '1px solid black', 'padding': '1px',
    'margin': '0 auto 0 auto'}
fig = gmaps.figure(layout=fig_layout,zoom_level=2,center=(15,25))
# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)
# Add layer
fig.add_layer(heatmap_layer)
fig.add_layer(hotel_layer)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…